In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tabgan.sampler import GANGenerator
import numpy as np


c:\Users\shrey\anaconda3\envs\torch4\Lib\site-packages\_ctgan\synthesizer.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.compat.v1.losses import mean_squared_error as old_mean_squared_error


In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [47]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

In [86]:
def preprocess_data(df):
    # Identify non-numeric columns
    non_numeric_cols = df.select_dtypes(exclude=['number', 'bool']).columns

    # Label Encode categorical columns
    le = LabelEncoder()
    df[non_numeric_cols] = df[non_numeric_cols].apply(lambda x: le.fit_transform(x))

    # Convert non-numeric columns back to integers
    df[non_numeric_cols] = df[non_numeric_cols].astype(int)

    # Convert boolean columns directly to integers
    bool_cols = df.select_dtypes(include='bool').columns
    df[bool_cols] = df[bool_cols].astype(int)
    return df

In [87]:

# Load your dataset
dataset_path = "data/cleaned/NF-UQ-NIDS-ATTACKS"
target_column = "Label"  # Replace with your target column name

df_pre = pd.read_csv(dataset_path, nrows=10000)
df = preprocess_data(df_pre)

# Specify the columns to be used
COLS_USED = list(df.columns)
COlS_TRAIN = list(df.columns).remove(target_column)


df = df[COLS_USED]
print(COLS_USED)


['Unnamed: 0', 'L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label', 'Attack', 'Dataset', 'sIP31', 'sIP30', 'sIP29', 'sIP28', 'sIP27', 'sIP26', 'sIP25', 'sIP24', 'sIP23', 'sIP22', 'sIP21', 'sIP20', 'sIP19', 'sIP18', 'sIP17', 'sIP16', 'sIP15', 'sIP14', 'sIP13', 'sIP12', 'sIP11', 'sIP10', 'sIP9', 'sIP8', 'sIP7', 'sIP6', 'sIP5', 'sIP4', 'sIP3', 'sIP2', 'sIP1', 'sIP0', 'dIP31', 'dIP30', 'dIP29', 'dIP28', 'dIP27', 'dIP26', 'dIP25', 'dIP24', 'dIP23', 'dIP22', 'dIP21', 'dIP20', 'dIP19', 'dIP18', 'dIP17', 'dIP16', 'dIP15', 'dIP14', 'dIP13', 'dIP12', 'dIP11', 'dIP10', 'dIP9', 'dIP8', 'dIP7', 'dIP6', 'dIP5', 'dIP4', 'dIP3', 'dIP2', 'dIP1', 'dIP0']


In [89]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(df.head())

   Unnamed: 0  L4_SRC_PORT  L4_DST_PORT  PROTOCOL  L7_PROTO  IN_BYTES  \
0          29        43025           25         6       3.0     41290   
1          40        64923           80         6       7.0       994   
2          67         9022          111         6      11.0       552   
3          93         1752           25         6       3.0      3022   
4         115        53967          111        17      11.0       168   

   OUT_BYTES  IN_PKTS  OUT_PKTS  TCP_FLAGS  FLOW_DURATION_MILLISECONDS  Label  \
0       2080       48        24         27                         803      1   
1        256       10         6         19                         189      1   
2        336       10         8         19                         678      1   
3       1636       20        20         19                         789      1   
4          0        2         0          0                           0      1   

   Attack  Dataset  sIP31  sIP30  sIP29  sIP28  sIP27  sIP26  sIP25  sIP24

In [90]:

# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop(target_column, axis=1),
    df[target_column],
    test_size=0.20,
    random_state=42
)

In [91]:

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

In [92]:
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

In [93]:
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [94]:
second_row_list = df.iloc[1].tolist()
print(second_row_list)

[40.0, 64923.0, 80.0, 6.0, 7.0, 994.0, 256.0, 10.0, 6.0, 19.0, 189.0, 1.0, 6.0, 3.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]


In [102]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                        patience=5, verbose=1, mode='auto',
                        restore_best_weights=True)

model.fit(x_train, y_train, validation_data=(x_test, y_test),
          callbacks=[monitor], verbose=2, epochs=10)

Epoch 1/10
69672/69672 - 81s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 81s/epoch - 1ms/step
Epoch 2/10
69672/69672 - 86s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 86s/epoch - 1ms/step
Epoch 3/10
69672/69672 - 77s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 77s/epoch - 1ms/step
Epoch 4/10
69672/69672 - 75s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 75s/epoch - 1ms/step
Epoch 5/10
69672/69672 - 76s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 76s/epoch - 1ms/step
Epoch 6/10
Restoring model weights from the end of the best epoch: 1.
69672/69672 - 78s - loss: 0.0000e+00 - val_loss: 0.0000e+00 - 78s/epoch - 1ms/step
Epoch 6: early stopping


In [105]:
gen_x, gen_y = GANGenerator().generate_data_pipe(df_x_train, df_y_train,
                                                  df_x_test, deep_copy=True,
                                                  only_adversarial=False,
                                                  use_adversarial=True)

TypeError: SampleData.generate_data_pipe() got an unexpected keyword argument 'gen_x_times'